## Gaussian Process Regression with a Tanimoto Kernel

The Tanimoto kernel operates on fingerprint representations of molecules

In [1]:
import sys
import os
sys.path.append('..')  # to import from GP.kernels and property_predition.data_utils

import gpflow
from gpflow.mean_functions import Constant
from gpflow.utilities import print_summary
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split

from GP.kernels import Tanimoto
from property_prediction.data_utils import transform_data, TaskDataLoader, featurise_mols


We load in SMILES from the photoswitch dataset

In [2]:
data_loader = TaskDataLoader('Photoswitch', '../datasets/photoswitches.csv')
smiles_list, y = data_loader.load_property_data()
print(smiles_list[0:5])

['C[N]1N=NC(=N1)N=NC2=CC=CC=C2', 'C[N]1C=NC(=N1)N=NC2=CC=CC=C2', 'C[N]1C=CC(=N1)N=NC2=CC=CC=C2', 'C[N]1C=C(C)C(=N1)N=NC2=CC=CC=C2', 'C[N]1C=C(C=N1)N=NC2=CC=CC=C2']


Featurise the SMILES in smiles_list into Morgan fingerprints. The featurise_mols function requires a list of SMILES and a specification of the representation, one of 'fingerprints', 'fragments' or 'fragprints'

In [3]:
X = featurise_mols(smiles_list, 'fingerprints')

In [4]:
# We define the Gaussian Process Regression training objective

m = None

def objective_closure():
    return -m.log_marginal_likelihood()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [6]:
#  We standardise the outputs but leave the inputs unchanged

_, y_train, _, y_test, y_scaler = transform_data(X_train, y_train, X_test, y_test)

In [7]:
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)

k = Tanimoto()
m = gpflow.models.GPR(data=(X_train, y_train), mean_function=Constant(np.mean(y_train)), kernel=k, noise_variance=1)

# Optimise the kernel variance and noise level by the marginal likelihood

opt = gpflow.optimizers.Scipy()
opt.minimize(objective_closure, m.trainable_variables, options=dict(maxiter=100))
print_summary(m)  # Model summary

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════╤═════════╤═══════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape   │ dtype   │     value │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════╪═════════╪═══════════╡
│ GPR.mean_function.c     │ Parameter │ Identity         │         │ True        │ ()      │ float64 │ 0.0847252 │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────┤
│ GPR.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()      │ float64 │ 0.492567  │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────┼─────────┼───────────┤
│ GPR.likelihood.variance │ Parameter │ Softplus + Shift │         │ True        │ ()      │ float64 │ 0.01552   │
╘═════════════════════════╧═══════════╧══════════════════╧═════════╧════════════

In [8]:
# mean and variance GP prediction

y_pred, y_var = m.predict_f(X_test)
y_pred = y_scaler.inverse_transform(y_pred)
y_test = y_scaler.inverse_transform(y_test)

In [9]:
# Compute R^2, RMSE and MAE on test set molecules

score = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

In [10]:
print("\nR^2: {:.3f}".format(score))
print("RMSE: {:.3f}".format(rmse))
print("MAE: {:.3f}".format(mae))


R^2: 0.932
RMSE: 18.001
MAE: 13.087
